In [ ]:
"""
https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html
"""

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [3]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read the book".split(), ["NN", "V", "DET", "NN"])
]

In [4]:
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)
tag_to_ix = {"DET":0, "NN":1, "V":2}

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'book': 7}


In [5]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

Create the model

In [28]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps form hidden state space to tag to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model

In [29]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the outputis the scoe for tag j for word i
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print(inputs)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([0, 1, 2, 3, 4])
tensor([[-1.4803, -0.9885, -0.9156],
        [-1.4962, -1.1197, -0.7993],
        [-1.5547, -1.0851, -0.7966],
        [-1.5691, -1.1254, -0.7609],
        [-1.5581, -1.1509, -0.7484]])


In [30]:
for epoch in range(300):    # again, normally you would NOT do 300 epochs, is is toy data/
    losses = 0.0
    sentences = 0
    avg_loss = 0.0
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

        sentences +=1
        losses += loss.item()

    if ((epoch+1) % 10) == 0:
        avg_loss = losses / sentences
        print(f"epoch {epoch+1}, loss: {avg_loss}")



epoch 10, loss: 1.0944843292236328
epoch 20, loss: 1.0541878938674927
epoch 30, loss: 1.0268694758415222
epoch 40, loss: 0.9989139437675476
epoch 50, loss: 0.9635710120201111
epoch 60, loss: 0.9154864847660065
epoch 70, loss: 0.8496389985084534
epoch 80, loss: 0.7628201544284821
epoch 90, loss: 0.6572480201721191
epoch 100, loss: 0.5426987111568451
epoch 110, loss: 0.43217869102954865
epoch 120, loss: 0.33551105856895447
epoch 130, loss: 0.2581239491701126
epoch 140, loss: 0.20061998069286346
epoch 150, loss: 0.1593463122844696
epoch 160, loss: 0.12967390567064285
epoch 170, loss: 0.10793327912688255
epoch 180, loss: 0.09161561727523804
epoch 190, loss: 0.07907107472419739
epoch 200, loss: 0.06921275332570076
epoch 210, loss: 0.06131218932569027
epoch 220, loss: 0.05487062782049179
epoch 230, loss: 0.04953860677778721
epoch 240, loss: 0.045065904036164284
epoch 250, loss: 0.04126979038119316
epoch 260, loss: 0.03801427595317364
epoch 270, loss: 0.03519636392593384
epoch 280, loss: 0.03

In [31]:
# See waht the scores are after training_data
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i.
    # The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)


tensor([[-0.0335, -3.9715, -4.2585],
        [-4.4422, -0.0333, -3.8654],
        [-3.7868, -4.3495, -0.0362],
        [-0.0198, -4.6475, -4.6038],
        [-4.7446, -0.0161, -4.9238]])


In [38]:
torch.max(tag_scores, 1)

torch.return_types.max(
values=tensor([-0.0335, -0.0333, -0.0362, -0.0198, -0.0161]),
indices=tensor([0, 1, 2, 0, 1]))